<a href="https://colab.research.google.com/github/PhilR1127/Data-601-Final-Project-/blob/main/TEAM_TBD_Robilotto_Thomas_Data_601_Final_Homework__12_17_21pjr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile

import pandas as pd

In [ ]:
## Use this if using the three datasets downloaded as csv files to hard drive

### from google.colab import files 
### upload = files.upload()

### NOTE - this method will not run all of the time unless you also upload the three files through this method
#### and requires significant time to upload the files - we also saw frequent runtime errors using this method, 
##### so we are using the zipped versions of the datasets from class (HW#3) in order to share our models so they will run 

In [ ]:
##Access the zipped data sets

!wget https://github.com/mguner/teaching/raw/main/datasets/MD_Crashes/Archive.zip



--2021-12-14 23:34:23--  https://github.com/mguner/teaching/raw/main/datasets/MD_Crashes/Archive.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mguner/teaching/main/datasets/MD_Crashes/Archive.zip [following]
--2021-12-14 23:34:24--  https://raw.githubusercontent.com/mguner/teaching/main/datasets/MD_Crashes/Archive.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51797738 (49M) [application/zip]
Saving to: ‘Archive.zip.1’

Archive.zip.1       100%[===================>]  49.40M   141MB/s    in 0.4s    

2021-12-14 23:34:24 (141 MB/s) - ‘Archive.zip.1’ saved [51797738/51797738]



In [ ]:
## extract and unzip the three files 
file_path= '/content/Archive.zip'
with ZipFile(file_path, 'r') as zip:
  zip.printdir() 
  zip.extractall(path='data') 


File Name                                             Modified             Size
MD_Statewide_Vehicle_Crashes_20-21_Anonymized_Person_Details.csv 2021-10-29 14:51:46     92251025
MD_Statewide_Vehicle_Crashes_20-21_Vehicle_Details.csv 2021-10-29 14:52:34     73439262
MS_Statewide_Vehicle_Crashes_20.csv            2021-10-29 14:53:18     61883538


In [ ]:
## Read the Zipped Files 
people = pd.read_csv('data/MD_Statewide_Vehicle_Crashes_20-21_Anonymized_Person_Details.csv', index_col = 0)

vehicles = pd.read_csv('data/MD_Statewide_Vehicle_Crashes_20-21_Vehicle_Details.csv', index_col = 0)

crashes = pd.read_csv('data/MS_Statewide_Vehicle_Crashes_20.csv', index_col = 0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
## Next we merged the "crashes" and the "people" data sets 
merged = pd.merge(crashes,people, on='REPORT_NO', how= 'outer', suffixes=['_crashes', '_people'])
merged.head(2)

,YEAR_crashes,QUARTER,LIGHT_DESC,LIGHT_CODE,COUNTY_DESC,COUNTY_NO,MUNI_DESC,MUNI_CODE,JUNCTION_DESC,JUNCTION_CODE,COLLISION_TYPE_DESC,COLLISION_TYPE_CODE,SURF_COND_DESC,SURF_COND_CODE,LANE_DESC,LANE_CODE,RD_COND_DESC,RD_COND_CODE,RD_DIV_DESC,RD_DIV_CODE,FIX_OBJ_DESC,FIX_OBJ_CODE,REPORT_NO,REPORT_TYPE,WEATHER_DESC,WEATHER_CODE,ACC_DATE,ACC_TIME,LOC_CODE,SIGNAL_FLAG_DESC,SIGNAL_FLAG,C_M_ZONE_FLAG,AGENCY_CODE,AREA_CODE,HARM_EVENT_DESC1,HARM_EVENT_CODE1,HARM_EVENT_DESC2,HARM_EVENT_CODE2,RTE_NO,ROUTE_TYPE_CODE,...,OCC_SEAT_POS_CODE,PED_VISIBLE_DESC,PED_VISIBLE_CODE,PED_OBEY_DESC,PED_OBEY_CODE,PED_TYPE_DESC,PED_TYPE_CODE,PED_LOCATION_CODE,MOVEMENT_DESC,MOVEMENT_CODE,PERSON_TYPE_DESC,PERSON_TYPE,ALCOHOL_TEST_DESC,ALCOHOL_TEST_CODE,ALCOHOL_TESTTYPE_DESC,ALCOHOL_TESTTYPE_CODE,DRUG_TEST_DESC,DRUG_TEST_CODE,DRUG_TESTRESULT_DESC,DRUG_TESTRESULT_CODE,BAC_CODE,FAULT_FLAG_DESC,FAULT_FLAG,EQUIP_PROB_DESC,EQUIP_PROB_CODE,SAF_EQUIP_DESC,SAF_EQUIP_CODE,EJECT_DESC,EJECT_CODE,DATE_OF_BIRTH,PERSON_ID,LICENSE_STATE_CODE,CLASS,CDL_FLAG_DESC,CDL_FLAG,VEHICLE_ID,EMS_UNIT_LABEL,AIRBAG_DEPLOYED,YEAR_people,Quarter
0,2020,Q2,Daylight,1.00,Baltimore,3.0,NaN,NaN,NaN,NaN,Other,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Applicable,0.0,AE58770052,Property Damage Crash,Not Applicable,0.00,20200618,15:15:00,NaN,No,N,N,BACOPD,UNK,Parked Vehicle,2.0,Not Applicable,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Driver,D,Not Applicable,0.0,NaN,NaN,Not Applicable,0.0,NaN,NaN,NaN,True,Y,Unknown,99.0,Unknown,99.0,Not Applicable,0.0,NaN,53715bb2-0082-4089-9a9e-62eabfd94dc8,NaN,NaN,False,False,3c9e8766-8b2a-4774-b85c-c645f2eabba0,NaN,0.0,2020.0,Q2
1,2020,Q2,NaN,6.02,Baltimore City,24.0,NaN,NaN,Non Intersection,1.0,Other,88,Dry,2.0,Left Turn Lane,2.0,No Defects,1.0,One-way Trafficway,2.0,Other Pole,10.0,ADJ984001J,Injury Crash,NaN,7.01,20200430,06:39:00,1003,Yes,Y,N,BALTIMORE,UNK,Other Vehicle,1.0,Other Vehicle,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Driver,D,Not Applicable,0.0,NaN,NaN,Not Applicable,0.0,NaN,NaN,NaN,False,N,No Misuse,1.0,Shoulder/Lap Belt(s),13.0,Not Ejected/Trapped,1.0,19750621.0,2a107168-e876-4624-be53-b123f41cf3ae,MD,NaN,False,False,6e506abc-74ce-46a2-b48c-2e09cdb1c20f,B,1.0,2020.0,Q2


In [ ]:
## Then we merged our new merged data set ('merged') with the third dataset "vehicles" 
merged_all = pd.merge(merged,vehicles, on='REPORT_NO', how ='outer', suffixes = ['_merged', '_vehicles'])

In [ ]:
## explore the new merged_all data set:
merged_all.head(2)

,YEAR_crashes,QUARTER,LIGHT_DESC,LIGHT_CODE,COUNTY_DESC,COUNTY_NO,MUNI_DESC,MUNI_CODE,JUNCTION_DESC,JUNCTION_CODE,COLLISION_TYPE_DESC,COLLISION_TYPE_CODE,SURF_COND_DESC,SURF_COND_CODE,LANE_DESC,LANE_CODE,RD_COND_DESC,RD_COND_CODE,RD_DIV_DESC,RD_DIV_CODE,FIX_OBJ_DESC,FIX_OBJ_CODE,REPORT_NO,REPORT_TYPE,WEATHER_DESC,WEATHER_CODE,ACC_DATE,ACC_TIME,LOC_CODE,SIGNAL_FLAG_DESC,SIGNAL_FLAG,C_M_ZONE_FLAG,AGENCY_CODE,AREA_CODE,HARM_EVENT_DESC1,HARM_EVENT_CODE1,HARM_EVENT_DESC2,HARM_EVENT_CODE2,RTE_NO,ROUTE_TYPE_CODE,...,VIN_NO,CV_BODY_TYPE_DESC,CV_BODY_TYPE_CODE,VEH_YEAR,VEH_MAKE,VEH_MODEL,COMMERCIAL_FLAG_DESC,COMMERCIAL_FLAG,HZM_NUM,TOWED_AWAY_FLAG_DESC,TOWED_AWAY_FLAG,NUM_AXLES,GVW_DESC,GVW_CODE,GOING_DIRECTION_DESC,GOING_DIRECTION_CODE,BODY_TYPE_DESC,BODY_TYPE_CODE,DRIVERLESS_FLAG_DESC,DRIVERLESS_FLAG,FIRE_FLAG_DESC,FIRE_FLAG,PARKED_FLAG_DESC,PARKED_FLAG,SPEED_LIMIT,HIT_AND_RUN_FLAG_DESC,HIT_AND_RUN_FLAG,HAZMAT_SPILL_FLAG_DESC,HAZMAT_SPILL_FLAG,VEHICLE_ID_vehicles,TOWED_VEHICLE_CONFIG_DESC,TOWED_VEHICLE_CONFIG_CODE,AREA_DAMAGED_CODE_IMP1,AREA_DAMAGED_CODE1,AREA_DAMAGED_CODE2,AREA_DAMAGED_CODE3,AREA_DAMAGED_CODE_MAIN_DESC,AREA_DAMAGED_CODE_MAIN,YEAR,Quarter_vehicles
0,2020,Q2,Daylight,1.0,Baltimore,3.0,NaN,NaN,NaN,NaN,Other,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Applicable,0.0,AE58770052,Property Damage Crash,Not Applicable,0.0,20200618,15:15:00,NaN,No,N,N,BACOPD,UNK,Parked Vehicle,2.0,Not Applicable,0.0,NaN,NaN,...,NMTKHMBX8KR092279,NaN,NaN,2019.0,TOY,C-HR,No,N,NaN,No,N,NaN,NaN,NaN,NaN,NaN,Passenger Car,2.0,Yes,Y,No,N,Yes,Y,5.0,No,N,NaN,NaN,1b3ba369-a3ef-4e2d-b47d-ebbb96d59917,Not Applicable,0.0,8.0,8.0,9.0,10.0,Eight o'clock,8.0,2020.0,Q2
1,2020,Q2,Daylight,1.0,Baltimore,3.0,NaN,NaN,NaN,NaN,Other,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Applicable,0.0,AE58770052,Property Damage Crash,Not Applicable,0.0,20200618,15:15:00,NaN,No,N,N,BACOPD,UNK,Parked Vehicle,2.0,Not Applicable,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,No,N,NaN,No,N,NaN,NaN,NaN,West,W,Unknown,99.0,No,N,No,N,No,N,5.0,Yes,Y,NaN,NaN,3c9e8766-8b2a-4774-b85c-c645f2eabba0,Not Applicable,0.0,1.0,1.0,NaN,NaN,One o'clock,1.0,2020.0,Q2


We were interested in exploring those accidents that resulted in severe injuries/fatalities. 

In [ ]:
##evaluate injury severity description
merged_all['INJ_SEVER_DESC'].value_counts()

No Injury                         519436
Non-incapacitating Injury          64242
Possible Incapacitating Injury     37177
Incapacitating/Disabled Injury      6997
Fatal Injury                        1085
Name: INJ_SEVER_DESC, dtype: int64

Wee see that the vast majority of vehicle accidents result in no injury.

But was there a relationship between fatalities and being ejected from the vehicle and/or certain age groups?  

***We hypothesized that younger age groups were more likley to be involved in fatal vehicle accidents and that fatal vehicle accidents were more likley to involve ejections (ejections being a proxy for lack of seat belt use).***  

***Our related business case proposition:  A seat belt campaign targeted towards younger age drivers would markedly reduce fatal accidents***

In [ ]:
#Next we looked at the jections descriptions of the accidents:
merged_all['EJECT_DESC'].value_counts()

Not Ejected/Trapped    545805
Not Applicable          60441
Unknown                  8972
Trapped                  4379
Fully Ejected            2250
Other                     554
Partially Ejected         396
Name: EJECT_DESC, dtype: int64

In [ ]:
## and who was injured/involved in these accidents.
merged_all['PERSON_TYPE_DESC'].value_counts()

Driver        485314
Occupant      137483
Pedestrian      6140
Name: PERSON_TYPE_DESC, dtype: int64

There are three categories of person types.... but only one Record No. and one Date of Birth for each row, so we assume these are associated with the driver. (*each accident involves a driver, but not necessarily an occuppant or a  pedestrian) *

In [ ]:
#.... and explored the columns in our merged_all dataframe:
merged_all.info(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 630088 entries, 0 to 630087
Data columns (total 151 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   YEAR_crashes                 int64  
 1   QUARTER                      object 
 2   LIGHT_DESC                   object 
 3   LIGHT_CODE                   float64
 4   COUNTY_DESC                  object 
 5   COUNTY_NO                    float64
 6   MUNI_DESC                    float64
 7   MUNI_CODE                    float64
 8   JUNCTION_DESC                object 
 9   JUNCTION_CODE                float64
 10  COLLISION_TYPE_DESC          object 
 11  COLLISION_TYPE_CODE          int64  
 12  SURF_COND_DESC               object 
 13  SURF_COND_CODE               float64
 14  LANE_DESC                    object 
 15  LANE_CODE                    float64
 16  RD_COND_DESC                 object 
 17  RD_COND_CODE                 float64
 18  RD_DIV_DESC                  object 
 19  R

In [ ]:
##Now we need to go back to the Date_Of_Birth column and transform that data into a usable format so we can actually calculuate the ages of the drivers.
### To do that we imported the parser from dateutil 
from dateutil import parser


In [ ]:
# After exploring relationships between multiple columns in the inividual three datasets (weather conditions, vehicle makes, injury severity codes, ejection status, alcohol codes etc),
## we decided to look at the the ages of the individuals involved in the accidents to see if we could find interesting realtionships relating to this new variable. 
### In order to do this we needed to create two columns (age_of_driver and accident_date) to datetime formats so they could be subtracted from each other 
#### to determine the actual ages of the individuals involved in the vehicle accidents. We initially used a method similar to what we used in homework #3 to obtain the year of the accident
  
merged_all['accident_date'] = pd.to_datetime(merged_all['ACC_DATE'], format = '%Y%m%d')
merged_all['accident_date']

0        2020-06-18
1        2020-06-18
2        2020-04-30
3        2020-04-30
4        2020-04-30
            ...    
630083   2021-06-24
630084   2021-06-01
630085   2021-06-01
630086   2021-06-01
630087   2021-06-01
Name: accident_date, Length: 630088, dtype: datetime64[ns]

In [ ]:
## We need to convert the birth data to a string format, then to date time so we could subtract the birth dates from the dates of the accidents. 
merged_all['DATE_OF_BIRTH'] = merged_all['DATE_OF_BIRTH'].astype('str')

In [ ]:
merged_all.info(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 630088 entries, 0 to 630087
Data columns (total 152 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   YEAR_crashes                 int64         
 1   QUARTER                      object        
 2   LIGHT_DESC                   object        
 3   LIGHT_CODE                   float64       
 4   COUNTY_DESC                  object        
 5   COUNTY_NO                    float64       
 6   MUNI_DESC                    float64       
 7   MUNI_CODE                    float64       
 8   JUNCTION_DESC                object        
 9   JUNCTION_CODE                float64       
 10  COLLISION_TYPE_DESC          object        
 11  COLLISION_TYPE_CODE          int64         
 12  SURF_COND_DESC               object        
 13  SURF_COND_CODE               float64       
 14  LANE_DESC                    object        
 15  LANE_CODE                    float64       
 16  R

We confirmed that we had increased the number of object dtypes in the merged_all dataframe by 1 (95 to 96) 

In [ ]:
## create new column "birthdates" in datetime format
merged_all['birthdates']=pd.to_datetime(merged_all['DATE_OF_BIRTH'], format='%Y%m%d')
merged_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 630088 entries, 0 to 630087
Columns: 153 entries, YEAR_crashes to birthdates
dtypes: datetime64[ns](2), float64(52), int64(3), object(96)
memory usage: 740.3+ MB


.... again confirming that we had added a new datetime64 dtype to our dataframe (increased from 2 to 3) 

In [ ]:
merged_all.head(2)

,YEAR_crashes,QUARTER,LIGHT_DESC,LIGHT_CODE,COUNTY_DESC,COUNTY_NO,MUNI_DESC,MUNI_CODE,JUNCTION_DESC,JUNCTION_CODE,COLLISION_TYPE_DESC,COLLISION_TYPE_CODE,SURF_COND_DESC,SURF_COND_CODE,LANE_DESC,LANE_CODE,RD_COND_DESC,RD_COND_CODE,RD_DIV_DESC,RD_DIV_CODE,FIX_OBJ_DESC,FIX_OBJ_CODE,REPORT_NO,REPORT_TYPE,WEATHER_DESC,WEATHER_CODE,ACC_DATE,ACC_TIME,LOC_CODE,SIGNAL_FLAG_DESC,SIGNAL_FLAG,C_M_ZONE_FLAG,AGENCY_CODE,AREA_CODE,HARM_EVENT_DESC1,HARM_EVENT_CODE1,HARM_EVENT_DESC2,HARM_EVENT_CODE2,RTE_NO,ROUTE_TYPE_CODE,...,CV_BODY_TYPE_CODE,VEH_YEAR,VEH_MAKE,VEH_MODEL,COMMERCIAL_FLAG_DESC,COMMERCIAL_FLAG,HZM_NUM,TOWED_AWAY_FLAG_DESC,TOWED_AWAY_FLAG,NUM_AXLES,GVW_DESC,GVW_CODE,GOING_DIRECTION_DESC,GOING_DIRECTION_CODE,BODY_TYPE_DESC,BODY_TYPE_CODE,DRIVERLESS_FLAG_DESC,DRIVERLESS_FLAG,FIRE_FLAG_DESC,FIRE_FLAG,PARKED_FLAG_DESC,PARKED_FLAG,SPEED_LIMIT,HIT_AND_RUN_FLAG_DESC,HIT_AND_RUN_FLAG,HAZMAT_SPILL_FLAG_DESC,HAZMAT_SPILL_FLAG,VEHICLE_ID_vehicles,TOWED_VEHICLE_CONFIG_DESC,TOWED_VEHICLE_CONFIG_CODE,AREA_DAMAGED_CODE_IMP1,AREA_DAMAGED_CODE1,AREA_DAMAGED_CODE2,AREA_DAMAGED_CODE3,AREA_DAMAGED_CODE_MAIN_DESC,AREA_DAMAGED_CODE_MAIN,YEAR,Quarter_vehicles,accident_date,birthdates
0,2020,Q2,Daylight,1.0,Baltimore,3.0,NaN,NaN,NaN,NaN,Other,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Applicable,0.0,AE58770052,Property Damage Crash,Not Applicable,0.0,20200618,15:15:00,NaN,No,N,N,BACOPD,UNK,Parked Vehicle,2.0,Not Applicable,0.0,NaN,NaN,...,NaN,2019.0,TOY,C-HR,No,N,NaN,No,N,NaN,NaN,NaN,NaN,NaN,Passenger Car,2.0,Yes,Y,No,N,Yes,Y,5.0,No,N,NaN,NaN,1b3ba369-a3ef-4e2d-b47d-ebbb96d59917,Not Applicable,0.0,8.0,8.0,9.0,10.0,Eight o'clock,8.0,2020.0,Q2,2020-06-18,NaT
1,2020,Q2,Daylight,1.0,Baltimore,3.0,NaN,NaN,NaN,NaN,Other,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Applicable,0.0,AE58770052,Property Damage Crash,Not Applicable,0.0,20200618,15:15:00,NaN,No,N,N,BACOPD,UNK,Parked Vehicle,2.0,Not Applicable,0.0,NaN,NaN,...,NaN,NaN,UNKNOWN,UNKNOWN,No,N,NaN,No,N,NaN,NaN,NaN,West,W,Unknown,99.0,No,N,No,N,No,N,5.0,Yes,Y,NaN,NaN,3c9e8766-8b2a-4774-b85c-c645f2eabba0,Not Applicable,0.0,1.0,1.0,NaN,NaN,One o'clock,1.0,2020.0,Q2,2020-06-18,NaT


In [ ]:
#determine the age of the indivuals involved in the vehicle accidents:
merged_all['age'] = merged_all['accident_date'] - merged_all['birthdates']



In [ ]:
## Need to format new columns in our merged_all dataframe: 1) year of birth, and 2) year of crash.. so they could be subtracted and we could calculate the ages of those individuals involved in the crashes
merged_all['year_of birth'] = (merged_all['birthdates']).dt.year
merged_all['year_of birth'].value_counts()

2000.0    15261
1998.0    14969
1999.0    14797
1997.0    14578
1996.0    14412
          ...  
1903.0        4
1914.0        4
1904.0        3
1907.0        2
1913.0        2
Name: year_of birth, Length: 122, dtype: int64

Our new 'year_of_birth' column demonstrates that the most common years of birth were late 1990s to 2000, which seems logical since we expect that most of the vehicle accidents in our database would involve younger drivers.  There were some suspicious outliers (ie 1903, 1904) but very few ( less than 5) ... so we will use this new column in our analysis. 

In [ ]:
# Next we obtained the year the actual crash occurred: 
merged_all['year_of_crash'] = (merged_all['accident_date']).dt.year
merged_all['year_of_crash'].value_counts()

2020    412489
2021    217599
Name: year_of_crash, dtype: int64

In [ ]:
## We then calculated the ages of the individuals involved in the accidents by subtracting the two new datetime objects:
merged_all['age_of_driver'] =merged_all['year_of_crash']-merged_all['year_of birth']


In [ ]:
## Check that the new 'age_of_driver" variable makes sense by looking at the most common values: 
merged_all['age_of_driver'].value_counts().nlargest(15)

21.0    15020
23.0    14852
22.0    14803
20.0    14751
24.0    14490
30.0    14488
29.0    14322
27.0    14300
25.0    14181
26.0    14118
28.0    14073
31.0    13927
32.0    13208
19.0    13073
33.0    12418
Name: age_of_driver, dtype: int64

We see that younger individuals were more likley to be involved in accidents.  The 15 most common ages to be involved in an accident were all between the ages of 19-33... which again makes sense to us and fits our original hypothesis.  We also decided to look at both the mean and median ages for our new variable.

In [ ]:
merged_all['age_of_driver'].mean()


37.64995031711177

In [ ]:
merged_all['age_of_driver'].median()

35.0

The mean age of the average individual was 37.6, while the mean was 35.  Now we can look at injury severity aggregated by the ages of involved individuals.  We expected to see younger ages (less than 35-37 years old) associated with more severe injury types; particularly fatalities:  

In [ ]:
# Look at the injury severities aggregated by the age of the driver (both by mean and median)
merged_agg2 =merged_all.groupby(['INJ_SEVER_DESC']).agg({'age_of_driver':'mean'})
merged_agg2

,age_of_driver
INJ_SEVER_DESC,
Fatal Injury,45.161290
Incapacitating/Disabled Injury,39.580210
No Injury,37.412715
Non-incapacitating Injury,37.895310
Possible Incapacitating Injury,39.604678


In [ ]:
merged_agg2 =merged_all.groupby(['INJ_SEVER_DESC']).agg({'age_of_driver':'median'})
merged_agg2

,age_of_driver
INJ_SEVER_DESC,
Fatal Injury,42.0
Incapacitating/Disabled Injury,36.0
No Injury,34.0
Non-incapacitating Injury,35.0
Possible Incapacitating Injury,36.0


This aggregation revealed an unexpected finding: We were suprised to see that it was actually older age indiviudals that were more likely to be involved in a vehicle accident involving a fatality (mean age of 45.6, median age of 42). 

We also expected to see that these fatalities would be associated with fully ejected ejection desciptions, so we created a new dataframe ('merged_agg3') grouped by both injury severity and ejection description, and then aggregated by the age of the individual involved in the vehicle accident. 


In [ ]:
merged_agg3 =merged_all.groupby(['INJ_SEVER_DESC','EJECT_DESC']).agg({'age_of_driver':'mean'})
merged_agg3

age_of_driver
INJ_SEVER_DESC                 EJECT_DESC                        
Fatal Injury                   Fully Ejected            37.231760
                               Not Applicable           42.235294
                               Not Ejected/Trapped      49.742972
                               Other                    33.000000
                               Partially Ejected        39.575758
                               Trapped                  47.035191
                               Unknown                  44.000000
Incapacitating/Disabled Injury Fully Ejected            35.137592
                               Not Applicable           36.665563
                               Not Ejected/Trapped      39.860571
                               Other                    39.840580
                               Partially Ejected        37.252874
                               Trapped                  42.824695
                               Unknown                  30.913043
No Injury                      Fully Ejected            37.233503
                               Not Applicable           38.118891
                               Not Ejected/Trapped      37.332681
                               Other                    39.279570
                               Partially Ejected        36.769231
                               Trapped                  41.345009
                               Unknown                  34.063353
Non-incapacitating Injury      Fully Ejected            35.971182
                               Not Applicable           38.371251
                               Not Ejected/Trapped      37.750000
                               Other                    40.028302
                               Partially Ejected        39.036364
                               Trapped                  43.555334
                               Unknown                  28.987261
Possible Incapacitating Injury Fully Ejected            36.908319
                               Not Applicable           39.433146
                               Not Ejected/Trapped      39.663490
                               Other                    43.714286
                               Partially Ejected        42.917910
                               Trapped                  43.050611
                               Unknown                  34.561404

In [ ]:
merged_agg3median =merged_all.groupby(['INJ_SEVER_DESC','EJECT_DESC']).agg({'age_of_driver':'median'})
merged_agg3median

age_of_driver
INJ_SEVER_DESC                 EJECT_DESC                        
Fatal Injury                   Fully Ejected                 33.0
                               Not Applicable                31.0
                               Not Ejected/Trapped           50.0
                               Other                         33.0
                               Partially Ejected             33.0
                               Trapped                       46.0
                               Unknown                       51.0
Incapacitating/Disabled Injury Fully Ejected                 31.5
                               Not Applicable                32.0
                               Not Ejected/Trapped           37.0
                               Other                         37.0
                               Partially Ejected             36.0
                               Trapped                       40.0
                               Unknown                       30.0
No Injury                      Fully Ejected                 32.0
                               Not Applicable                35.0
                               Not Ejected/Trapped           34.0
                               Other                         38.5
                               Partially Ejected             35.0
                               Trapped                       40.0
                               Unknown                       29.5
Non-incapacitating Injury      Fully Ejected                 32.0
                               Not Applicable                35.0
                               Not Ejected/Trapped           35.0
                               Other                         39.0
                               Partially Ejected             39.0
                               Trapped                       42.0
                               Unknown                       28.0
Possible Incapacitating Injury Fully Ejected                 33.0
                               Not Applicable                36.0
                               Not Ejected/Trapped           36.0
                               Other                         48.0
                               Partially Ejected             39.0
                               Trapped                       40.0
                               Unknown                       31.0

We were again somewhat suprised by our findings: 

We unexpectedly saw that it was actually an "average" age group that suffered fatalities involving full ejections.... while we found that older age fatalities tended to be associated with the victims being trapped/not ejected.  

We decided to explore this further to see if we could find any associations with vehicle makes or safety equipment misuse

In [ ]:
## It was diffcult to find an obvious realtionship between the ejection descriptions and safety equipment use
merged_all.groupby(['EJECT_DESC','EQUIP_PROB_DESC']).size().nlargest(25)

EJECT_DESC           EQUIP_PROB_DESC   
Not Ejected/Trapped  No Misuse             389950
                     Not Applicable        105269
                     Unknown                49298
Not Applicable       Not Applicable         35632
                     No Misuse              17051
                     Unknown                 7668
Unknown              Unknown                 7094
Trapped              No Misuse               3015
Unknown              Not Applicable          1509
Trapped              Not Applicable          1073
Fully Ejected        Not Applicable          1070
                     No Misuse               1024
Not Ejected/Trapped  Air Bag Failed           370
Unknown              No Misuse                359
Not Ejected/Trapped  Other                    339
Other                Not Applicable           302
Trapped              Unknown                  255
Not Ejected/Trapped  Belt(s) Misused          236
Partially Ejected    Not Applicable           196
Other     

In [ ]:
## aggregating by median age did also not reveal any initially obvious patterns:
merged_all.groupby(['EJECT_DESC','EQUIP_PROB_DESC']).agg({'age_of_driver':'median'})

age_of_driver
EJECT_DESC          EQUIP_PROB_DESC                    
Fully Ejected       Air Bag Failed                 30.0
                    Belt(s) Misused                 2.0
                    Belts/Anchors Broken           24.0
                    No Misuse                      35.0
                    Not Anchored Right              1.0
                    Not Applicable                 30.0
                    Not Strapped Right             13.5
                    Other                          52.0
                    Size/Type Improper             37.5
                    Strap/Tether Loose             33.0
                    Unknown                        31.0
Not Applicable      Air Bag Failed                 28.0
                    Belt(s) Misused                52.0
                    Facing Wrong Way               26.0
                    No Misuse                      36.0
                    Not Applicable                 35.0
                    Not Strapped Right             20.0
                    Other                          37.0
                    Size/Type Improper              0.0
                    Strap/Tether Loose              0.0
                    Unknown                        34.0
Not Ejected/Trapped Air Bag Failed                 33.0
                    Anchor Not Secure              22.5
                    Belt(s) Misused                25.0
                    Belts/Anchors Broken           24.0
                    Facing Wrong Way               27.0
                    No Misuse                      35.0
                    Not Anchored Right              4.0
                    Not Applicable                 34.0
                    Not Strapped Right              9.0
                    Other                          30.0
                    Size/Type Improper              6.0
                    Strap/Tether Loose             25.0
                    Unknown                        33.0
Other               Anchor Not Secure               1.0
                    Belt(s) Misused                 2.0
                    No Misuse                      46.0
                    Not Applicable                 42.0
                    Not Strapped Right              2.0
                    Other                          23.0
                    Unknown                        35.0
Partially Ejected   No Misuse                      40.0
                    Not Applicable                 35.5
                    Not Strapped Right              2.0
                    Unknown                        36.0
Trapped             Air Bag Failed                 29.5
                    Belt(s) Misused                29.0
                    Belts/Anchors Broken           37.0
                    No Misuse                      42.0
                    Not Anchored Right             25.0
                    Not Applicable                 38.0
                    Unknown                        40.0
Unknown             Air Bag Failed                  NaN
                    Belt(s) Misused                 NaN
                    No Misuse                      35.0
                    Not Applicable                 30.0
                    Other                          38.0
                    Unknown                        27.0

In [ ]:
# Next we looked at potentail relationships with vehicle makes. First we needed to understand the most common vehicle types in the merged_all data set
merged_all['VEH_MAKE'].value_counts()

FORD                 73474
HONDA                62928
TOYOTA               60882
NISSAN               42845
UNKNOWN              29342
                     ...  
DAKO                     1
CADALIC                  1
JTKDE167970210127        1
INIFNITI                 1
CHECROLET                1
Name: VEH_MAKE, Length: 3129, dtype: int64

We see that the most common types of vehicles in our dataset were: 
1) Ford, 2) Honda, and 3) Toyota

We further aggregated our data by creating the merg_agg4 dataframe, which we aggregated by mean age of driver and vehicle make value counts.  We found only that the two most common types of vehicle makes in the dataset (Ford, Honda) were the two most common types of makes associated with fatalities involving trapped, older individuals.


In [ ]:
merged_agg4 =merged_all.groupby(['INJ_SEVER_DESC','EJECT_DESC','VEH_MAKE']).agg({'age_of_driver':'mean','VEH_MAKE':'count'})
merged_agg4.info

<bound method DataFrame.info of                                                        age_of_driver  VEH_MAKE
INJ_SEVER_DESC                 EJECT_DESC    VEH_MAKE                         
Fatal Injury                   Fully Ejected ACURA         26.000000         2
                                             APOLLO        39.000000         1
                                             AUDI          43.333333         3
                                             BMW           49.000000         1
                                             BUICK         25.000000         1
...                                                              ...       ...
Possible Incapacitating Injury Unknown       TOYT          29.000000         1
                                             UNK           21.000000         1
                                             UNKNOWN       26.750000        13
                                             XX            44.000000         1
                    

In [ ]:
merged_agg4[(merged_agg4.VEH_MAKE>40)]
#merged_agg4[(merged_agg4.age_of_driver>45)]

age_of_driver  VEH_MAKE
INJ_SEVER_DESC                 EJECT_DESC    VEH_MAKE                                
Fatal Injury                   Trapped       FORD                 49.094340        53
                                             HONDA                46.465116        43
Incapacitating/Disabled Injury Fully Ejected FORD                 35.901639        62
                                             HARLEY DAVIDSON      47.300000        70
                                             HONDA                32.000000       100
...                                                                     ...       ...
Possible Incapacitating Injury Trapped       CHEVY                42.600000        55
                                             FORD                 43.863636       154
                                             HONDA                39.787879        99
                                             NISSAN               42.463768        69
                                             TOYOTA               41.780303       132

[536 rows x 2 columns]

**FINDINGS: **
1) Younger aged individuals (top 15 most frequent ages: ages 19-33; median age 35) were most likley to be involved in a motor vehicle accident. 

2) Older aged individuals were (suprisingly) more likely to be involved in fatal accidents (median age 42, mean age 45)  

3) Interestingly, younger to average age fatalities (mean 37.2/median 33) were associated with fully ejected status of the victim , while those fatalities associated with an older age group (ages groups mean/median 46- 50) invoved a trapped and trapped/not ejected status. 

**SO - WOULD A CAMPAIGN TARGETING SAFETY BELT SAFETY IN YOUNGER AGE DRIVERS SAVE LIVES? **

**......Probably not....**

**CONCLUSIONS/RECOMENDATIONS:** 

1) We hypothesize that younger to average age fatalities (fully ejected status) are due to those individuals not wearing seat belts or wearing them incorrectly. 

2)Since fatalities involving ejected vehicle occupants were seen with a typical /average age of drivers involved in all accidents we can't recommend our original business proposal of targeting only younger age groups in an aggressive seat belt campaign.  

3)We recommend continuing to explore older age vehicle accident fatality data. Why are older age drivers more commonly involved in fatal accidents?... and why they are associated with a trapped/not ejected status?: (Possible associations with time of day, specific locations, weather, vehicle age, visual acuity, access to cell phone etc)  - in order to identify modifiable factors that could be used to help reduce the risk of older individuals being trapped in their cars and suffering fatalities during a motor vehicle accident. 
